In [46]:
import os
import numpy as np

from flow.scenarios.merge import MergeScenario, ADDITIONAL_NET_PARAMS
from flow.core.params import VehicleParams, NetParams, InitialConfig, TrafficLightParams, InFlows, SumoParams, EnvParams, SumoCarFollowingParams
from flow.envs.merge import WaveAttenuationMergePOEnv, ADDITIONAL_ENV_PARAMS as WA_ADDITIONAL_ENV_PARAMS
from flow.envs.loop.loop_accel import AccelEnv, ADDITIONAL_ENV_PARAMS as LA_ADDITIONAL_ENV_PARAMS
from flow.core.experiment import Experiment

In [47]:
%load_ext autoreload
%autoreload 2
from merge import NashController 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Add Vehicles to the Scenario
These vehicle will be controlled by whatever acceleration controller set to.

In [48]:
name = "merge_example"
vehicles = VehicleParams()
vehicles.add("human",
             acceleration_controller=(NashController, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="aggressive"))

## Set the Net Params
These control the configuration of the highway merge network.

In [49]:
# Optionally adjust the net params
additional_net_params = ADDITIONAL_NET_PARAMS.copy()
additional_net_params['merge_length'] = 50 # length of the merge edge
additional_net_params['pre_merge_length'] = 50 # length of the highway leading to the merge
additional_net_params['post_merge_length'] = 100  # length of the highway past the merge
additional_net_params['merge_lanes'] = 1 # number of lanes in the merge
additional_net_params['highway_lanes'] = 1 # number of lanes in the highway
additional_net_params['speed_limit'] = 30 # max speed limit of the network

## Set the Environment Params
We can use the WaveAttenuationEnv or the AccelEnv.

In [50]:
# Optionally adjust the env params
USE_WA_ENV = False

additional_env_params = None
if USE_WA_ENV:
    additional_env_params = WA_ADDITIONAL_ENV_PARAMS.copy()
    additional_env_params['max_accel'] = 3
    additional_env_params['max_decel'] = 3
    additional_env_params['target_velocity'] = 25
    additional_env_params['num_rl'] = 5
else:
    additional_env_params = LA_ADDITIONAL_ENV_PARAMS.copy()
    additional_env_params['max_accel'] = 3
    additional_env_params['max_decel'] = 3
    additional_env_params['target_velocity'] = 25
    additional_env_params['sort_vehicles'] = False

env_params = EnvParams(additional_params=additional_env_params,
                       horizon=float('inf'),
                       warmup_steps=0,
                       sims_per_step=1,
                       evaluate=False,
                       clip_actions=True)

## Add Inflows to the Scenario

In [51]:
inflow = InFlows()

# Inflow for highway lanes
inflow.add(veh_type="human",
           edge="inflow_highway",
           probability=1.0, #0.5
           begin=1,
           number=2, # 100
           depart_speed=additional_env_params['target_velocity'],
           color="red")

# Inflow for merge lane
inflow.add(veh_type="human",
           edge="inflow_merge",
           probability=1.0, #0.1
           depart_speed="random", #additional_env_params['target_velocity'],
           begin=1, 
           number=1, # 30
           color="blue")

net_params = NetParams(inflows=inflow,
                       additional_params=additional_net_params)

## Set the Initial Configuration Params
These parameters that affect the positioning of vehicle in the network at the start of a rollout. By default, vehicles are uniformly distributed in the network.

In [52]:
initial_config = InitialConfig(shuffle=False,
                               spacing="uniform",
                               min_gap=0,
                               perturbation=1,
                               x0=0,
                               bunching=0,
                               lanes_distribution=float("inf"),
                               edges_distribution="all",
                               additional_params=None)

## Set the Traffic Light Params
This class is used to place traffic lights in the network and describe the state of these traffic lights. In addition, this class supports modifying the states of certain lights via TraCI.

In [53]:
traffic_lights = TrafficLightParams()

## Set the Simulation Params
These include the simulation time step, whether or not to render the simulation, and where to write the output results.

In [54]:
sumo_params = SumoParams(sim_step=0.1, render=True, emission_path='data')

## Run the Experiment

In [55]:
# create the scenario object
scenario = MergeScenario(name="merge_example",
                         vehicles=vehicles,
                         net_params=net_params,
                         initial_config=initial_config,
                         traffic_lights=traffic_lights)

# create the environment object
env = None
if USE_WA_ENV:
    env = WaveAttenuationMergePOEnv(env_params, sumo_params, scenario)
else:
    env = AccelEnv(env_params, sumo_params, scenario)

# create the experiment object
exp = Experiment(env)

# run the experiment for a set number of rollouts / time steps
_ = exp.run(1, 3000, convert_to_csv=True)

**********************************************************
**********************************************************
**********************************************************
significantly decrease after large number of rollouts. In 
order to avoid this, set SumoParams(restart_instance=True).
**********************************************************
**********************************************************
**********************************************************
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      152
Number of nonzeros in inequality constraint Jacobian.:       40
Number of nonzeros in Lagrangian Hessian.............:     1830

Total number of variables............................:       60
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0

/home/jacob/anaconda3/envs/flow/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/jacob/anaconda3/envs/flow/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



  19  8.7852396e+03 2.84e-14 2.87e-04  -5.7 3.93e-02    -  1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20  1.7367220e+03 2.84e-14 8.46e-05  -5.7 2.61e-02    -  1.00e+00 1.00e+00f  1
  21  3.4390250e+02 2.84e-14 2.51e-05  -5.7 1.74e-02    -  1.00e+00 1.00e+00f  1
  22  6.8508061e+01 2.84e-14 7.43e-06  -5.7 1.15e-02    -  1.00e+00 1.00e+00f  1
  23  1.3550537e+01 2.84e-14 2.20e-06  -8.6 9.41e-03    -  9.98e-01 1.00e+00f  1
  24  2.6813635e+00 2.84e-14 6.56e-07  -8.6 1.64e-02    -  1.00e+00 1.00e+00h  1
  25  5.3211563e-01 2.84e-14 1.98e-07  -8.6 5.28e-03    -  1.00e+00 1.00e+00h  1
  26  1.0594182e-01 2.84e-14 5.91e-08  -8.6 9.69e-03    -  1.00e+00 1.00e+00h  1
  27  2.1102631e-02 2.84e-14 1.72e-08  -8.6 5.52e-03    -  1.00e+00 1.00e+00h  1
  28  4.1353379e-03 2.84e-14 4.94e-09 -12.9 4.83e-03    -  9.98e-01 1.00e+00h  1
  29  7.9361894e-04 2.84e-14 1.39e-09 -12.9 4.95e-03    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 

   6  1.0735637e+13 2.84e-14 1.92e+03  -1.0 7.34e+00    -  1.00e+00 1.00e+00f  1
   7  2.1206247e+12 2.84e-14 5.68e+02  -1.0 4.89e+00    -  1.00e+00 1.00e+00f  1
   8  4.1889253e+11 2.84e-14 1.68e+02  -1.0 3.26e+00    -  1.00e+00 1.00e+00f  1
   9  8.2746893e+10 2.84e-14 4.98e+01  -1.0 2.18e+00    -  1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  1.6346971e+10 2.84e-14 1.48e+01  -1.0 1.45e+00    -  1.00e+00 1.00e+00f  1
  11  3.2303674e+09 2.84e-14 4.38e+00  -1.0 9.67e-01    -  1.00e+00 1.00e+00f  1
  12  6.3904612e+08 2.84e-14 1.30e+00  -1.0 6.44e-01    -  1.00e+00 1.00e+00f  1
  13  1.2694973e+08 2.84e-14 3.82e-01  -1.0 4.26e-01    -  1.00e+00 1.00e+00f  1
  14  2.5183154e+07 2.84e-14 1.14e-01  -1.7 2.86e-01    -  1.00e+00 1.00e+00f  1
  15  4.9851773e+06 2.84e-14 3.39e-02  -2.5 1.93e-01    -  1.00e+00 1.00e+00f  1
  16  9.9219777e+05 2.84e-14 9.99e-03  -2.5 1.27e-01    -  1.00e+00 1.00e+00f  1
  17  1.9631546e+05 2.84e-14

   1  3.4360494e+16 1.42e-14 8.17e+05  -1.0 5.35e+01    -  1.19e-03 9.33e-01f  1
   2  6.7872590e+15 1.42e-14 2.42e+05  -1.0 3.69e+01    -  5.98e-01 1.00e+00f  1
   3  1.3406934e+15 1.42e-14 7.16e+04  -1.0 2.46e+01    -  9.45e-01 1.00e+00f  1
   4  2.6482839e+14 1.42e-14 2.12e+04  -1.0 1.64e+01    -  1.00e+00 1.00e+00f  1
   5  5.2311793e+13 1.42e-14 6.29e+03  -1.0 1.09e+01    -  1.00e+00 1.00e+00f  1
   6  1.0333200e+13 1.42e-14 1.86e+03  -1.0 7.28e+00    -  1.00e+00 1.00e+00f  1
   7  2.0411309e+12 2.84e-14 5.52e+02  -1.0 4.85e+00    -  1.00e+00 1.00e+00f  1
   8  4.0319006e+11 2.84e-14 1.64e+02  -1.0 3.24e+00    -  1.00e+00 1.00e+00f  1
   9  7.9645175e+10 2.84e-14 4.85e+01  -1.0 2.16e+00    -  1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  1.5734289e+10 2.84e-14 1.44e+01  -1.0 1.44e+00    -  1.00e+00 1.00e+00f  1
  11  3.1093460e+09 2.84e-14 4.26e+00  -1.0 9.59e-01    -  1.00e+00 1.00e+00f  1
  12  6.1514295e+08 2.84e-14

...

  82  1.4238499e+11 1.42e-14 3.00e+01  -1.0 6.33e+01    -  1.87e-01 8.43e-02f  1
  83  1.1691052e+11 1.42e-14 2.61e+01  -1.0 6.07e+01    -  2.95e-01 1.80e-01f  1
  84  8.9778038e+10 7.11e-15 2.22e+01  -1.0 6.19e+01    -  2.23e-02 2.38e-01f  1
  85  5.9234880e+10 1.42e-14 1.81e+01  -1.0 6.26e+01    -  3.66e-01 3.27e-01f  1
  86  1.3822510e+10 7.11e-15 1.97e+01  -1.0 5.59e+01    -  2.96e-01 1.00e+00f  1
  87  3.1622217e+09 1.42e-14 3.96e+00  -1.0 4.77e+01    -  8.11e-01 9.31e-01f  1
  88  6.3367318e+08 1.42e-14 1.06e+00  -1.0 3.76e+01    -  5.14e-01 1.00e+00f  1
  89  1.5952971e+08 7.11e-15 3.23e-01  -1.0 2.48e+01    -  1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  90  3.9796671e+07 7.11e-15 1.01e-01  -1.7 1.68e+01    -  1.00e+00 1.00e+00h  1
  91  9.8276004e+06 1.42e-14 3.18e-02  -2.5 1.25e+01    -  1.00e+00 1.00e+00h  1
  92  3.0267834e+06 1.42e-14 1.06e-02  -2.5 8.75e+00    -  1.00e+00 1.00e+00h  1
  93  1.7097992e+06 7.11e-15

  42  6.6674904e+15 8.59e+01 1.83e+05  -1.0 2.63e+02    -  1.49e-04 2.12e-03f  1
  43  6.4487742e+15 8.36e+01 1.78e+05  -1.0 2.63e+02    -  4.27e-04 2.54e-02f  1
  44  6.4458390e+15 8.36e+01 1.78e+05  -1.0 2.56e+02    -  8.26e-03 3.59e-04f  1
  45  6.4292913e+15 8.34e+01 1.77e+05  -1.0 2.56e+02    -  5.71e-04 2.09e-03f  1
  46  6.3777565e+15 8.29e+01 1.76e+05  -1.0 2.55e+02    -  1.00e-04 6.47e-03f  1
  47  6.1774899e+15 8.08e+01 1.72e+05  -1.0 2.54e+02    -  1.38e-03 2.46e-02f  1
  48  6.1748775e+15 8.07e+01 1.72e+05  -1.0 2.47e+02    -  1.09e-02 3.38e-04f  1
  49  6.1430010e+15 8.04e+01 1.71e+05  -1.0 2.47e+02    -  1.74e-04 4.22e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  50  5.9440687e+15 7.83e+01 1.67e+05  -1.0 2.46e+02    -  1.38e-03 2.56e-02f  1
  51  5.9419000e+15 7.83e+01 1.67e+05  -1.0 2.40e+02    -  2.20e-02 2.90e-04f  1
  52  5.9255855e+15 7.81e+01 1.66e+05  -1.0 2.40e+02    -  2.79e-04 2.33e-03f  1
  53  5.6997040e+15 7.56e+01

 152  1.8223493e+14 9.14e-01 3.84e+03  -1.0 2.34e+01    -  1.44e-02 8.86e-04f  1
 153  1.8049540e+14 8.87e-01 3.73e+03  -1.0 1.89e+01    -  1.55e-04 2.91e-02f  1
 154  1.7678681e+14 8.43e-01 3.55e+03  -1.0 2.23e+02    -  2.79e-03 4.91e-02f  1
 155  1.7675124e+14 8.43e-01 3.54e+03  -1.0 2.25e+02    -  7.48e-02 4.91e-04f  1
 156  1.7657326e+14 8.41e-01 3.53e+03  -1.0 1.23e+02    -  3.92e-03 2.88e-03f  1
 157  1.7483969e+14 8.17e-01 3.43e+03  -1.0 1.23e+02    -  1.26e-03 2.83e-02f  1
 158  1.7480571e+14 8.16e-01 3.43e+03  -1.0 3.13e+02    -  9.84e-04 4.87e-04f  1
 159  1.7480212e+14 8.16e-01 3.43e+03  -1.0 2.88e+02    -  2.77e-02 5.66e-05f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 160  1.7451073e+14 8.13e-01 3.42e+03  -1.0 2.82e+02    -  1.76e-03 4.62e-03f  1
 161  1.7372105e+14 8.02e-01 3.38e+03  -1.0 2.87e+02    -  8.41e-04 1.24e-02f  1
 162  1.7310521e+14 7.97e-01 3.35e+03  -1.0 3.13e+02    -  9.15e-04 7.14e-03f  1
 163  1.7309518e+14 7.97e-01

 264  1.6922644e+06 1.42e-14 2.26e-03  -2.5 1.60e+01    -  1.00e+00 1.00e+00h  1
 265  5.3950305e+05 1.42e-14 2.09e-03  -3.8 7.35e+00    -  1.00e+00 1.00e+00h  1
 266  1.8000718e+05 1.42e-14 8.33e-04  -3.8 5.21e+00    -  1.00e+00 1.00e+00h  1
 267  8.9740890e+04 7.11e-15 1.69e-04  -3.8 2.77e+00    -  1.00e+00 1.00e+00h  1
 268  2.7608457e+04 7.11e-15 2.51e-04  -5.7 3.09e+00    -  1.00e+00 1.00e+00h  1
 269  7.7970439e+03 1.42e-14 1.17e-04  -5.7 2.52e+00    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 270  2.4267846e+03 7.11e-15 3.99e-05  -5.7 1.74e+00    -  1.00e+00 1.00e+00h  1
 271  1.1350092e+03 1.42e-14 8.00e-06  -5.7 8.89e-01    -  1.00e+00 1.00e+00h  1
 272  9.3149485e+02 1.42e-14 3.68e-07  -5.7 1.99e-01    -  1.00e+00 1.00e+00h  1
 273  2.9493674e+02 7.11e-15 7.97e-06  -8.6 1.01e+00    -  1.00e+00 1.00e+00h  1
 274  8.0724901e+01 1.42e-14 4.12e-06  -8.6 8.62e-01    -  1.00e+00 1.00e+00h  1
 275  2.1129174e+01 7.11e-15

FatalTraCIError: connection closed by SUMO

In [ ]:
emission_location = os.path.join(exp.env.sim_params.emission_path, exp.env.scenario.name)
print(emission_location + '-emission.csv')